# Technical test results for HEVA company

This notebook repeats the statement of the test. Under each activity you will find the code and the result produced.
You will find all the requirements to run this notebook in the requirements.md file.